In [56]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt

import CBEngine
import json
import traceback
import argparse
import logging
import os
import sys
import time
from pathlib import Path
import re
import gym
import numpy as np
from datetime import datetime
import agent.gym_cfg as gym_cfg
from agent.utils import PreprocEnv,get_args

from agent.agent_my import agent_specs
from train_dqn_example import read_config, process_roadnet, process_delay_index, run_simulation

import torch
import numpy as np
import scipy.signal
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from torch.distributions.categorical import Categorical
from torch.optim import Adam


from core import combined_shape, mlp, count_vars, discount_cumsum, VPGBuffer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [366]:
simulator_cfg_file = './cfg/simulator.cfg'

gym_configs = gym_cfg.gym_cfg().cfg
simulator_configs = read_config(simulator_cfg_file)
env = gym.make(
    'CBEngine-v0',
    simulator_cfg_file=simulator_cfg_file,
    thread_num=16,
    gym_dict=gym_configs
)
scenario = [
    'test'
]


roadnet_path = Path(simulator_configs['road_file_addr'])
intersections, roads, agents = process_roadnet(roadnet_path)
env.set_log(0)
env.set_warning(0)
args = get_args()

In [381]:
def compute_loss_v(data):
    obs, ret = data['obs'], data['ret']
    v_new = []
    for i in range(act.shape[0]):
        v_new.append(agent.v(agent.feat_prepr(obs[i])))
    return ((torch.stack(v_new) - ret)**2).mean()

def compute_loss_pi(data):
    obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']
  
    ar_logp = []
    for i in range(act.shape[0]):
        pi, logp  = agent.pi(agent.feat_prepr(obs[i]), act[i])
        ar_logp.append(logp)
    ar_logp = torch.stack(ar_logp)    
    # Policy loss
    loss_pi = -(ar_logp * adv).mean()
    # Useful extra info
    approx_kl = (logp_old - ar_logp).mean().item()
    ent = pi.entropy().mean().item()
    pi_info = dict(kl=approx_kl, ent=ent) 
    return loss_pi, pi_info

def update():
    data = buf.get()

    # Train policy with a single step of gradient descent
    pi_optimizer.zero_grad()
    loss_pi, pi_info = compute_loss_pi(data)
    loss_pi.backward( )
    pi_optimizer.step()

    # Value function learning
    train_v_iters = 10 # wtf!!!!!!!!!!!!!!!!!!!  
#     for i in range(train_v_iters):
    vf_optimizer.zero_grad()
    loss_v = compute_loss_v(data)
    loss_v.backward( )#     retain_graph=True
    vf_optimizer.step()

In [370]:
args.action_interval

4

In [399]:
buf = VPGBuffer((22,24), (agent.n_agnts,), local_steps_per_epoch, args.gamma)

o = env.reset()[0]

agent_id_list = []
for k in o:
    agent_id_list.append(int(k.split('_')[0]))
agent_id_list = list(set(agent_id_list))
agent = agent_specs[scenario[0]]
agent.load_agent_list(agent_id_list )#!!!!!!!!!!!!!!!!!
agent.get_data_from_env(env) #!!!!!!!!!!!!!!!!!

local_steps_per_epoch = 20

pi_optimizer = Adam(agent.pi.parameters(), lr=0.001)
vf_optimizer = Adam(agent.v.parameters(), lr=0.001)

def make_step(j, act):  
    rwd_t = 0
    for _ in range(args.action_interval): 
        oo, _, d, _ = agent.env_preproc.step(env, act) # simulates 10 sec
        j += 1
        rwd_t += oo
        done = all(d.values())
        if done:
            print('all done')
            break
    rwd_t = rwd_t / args.action_interval    
    return j, done, rwd_t

# for e in range(args.episodes):
Nepochs  = 100
for e in range(Nepochs):
    rw_average = []
    o = agent.env_preproc.preproc_obs(env.reset()[0])
    i = 0
    
    for t in range(local_steps_per_epoch):  
#         print('___',o.view((-1,2,4,3)).permute([0,1,3,2]).reshape((-1,6,4)))
        a, v, logp = agent.step(o)
        # We keep the same action for a certain time          
        i, don, next_o = make_step(i, a)
#         print(i, a)
        rw = next_o[:, 13: 25].sum(dim=1)-next_o[:, 1: 13].sum(dim=1)
#         print(buf.ptr)
        buf.store(o, a, rw.mean(), v, logp)
        rw_average.append(rw.mean())
        # Update obs (critical!)
        o = next_o
        if don or i>int(args.steps/10)-args.action_interval-1:
            print(don, i)
            break         

    v = 0
    if not don:
        _, v, _ = agent.step(o)        
    buf.finish_path(v)
    
    update()

    agent.pi.logits_net.reset_state()
    agent.v.v_net.reset_state()
    
    print("ep:{}/{}, average tt:{} , mean rwd: {}".format(e, Nepochs, env.eng.get_average_travel_time(), np.mean(rw_average)))

    
#     scores = run_simulation(agent_specs, simulator_cfg_file, gym_cfg)
#     print('total_served_vehicles ', scores[0],'  ;delay_index ', scores[1])

{}
{}
{0: {'distance': [12.0], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [6.0], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [118.88888888888891], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [230.00000000000006], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [322.0], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [0.0], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [322.0], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [0.0], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [322.0], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [0.0], 'start_time': [26.0], 't_ff': [77.976]}}
{0: {'distance': [322.0

{0: {'distance': [241.11111111111117], 'drivable': [7100.0], 'road': [71.0], 'route': [71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [26.0], 't_ff': [77.976]}, 1: {'distance': [207.77777777777783], 'drivable': [7100.0], 'road': [71.0], 'route': [71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [86.0], 't_ff': [77.976]}, 2: {'distance': [455.0], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [90.0], 't_ff': [104.208]}, 3: {'distance': [283.0], 'drivable': [9901.0], 'road': [99.0], 'route': [99.0, 15.0], 'speed': [0.0], 'start_time': [91.0], 't_ff': [62.10000000000001]}, 4: {'distance': [122.22222222222224], 'drivable': [7100.0], 'road': [71.0], 'route': [71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [146.0], 't_ff': [77.976]}, 5: {'distance': [287.0], 'drivable': [1201.0], 'road': [12.0], 'route': [12.0, 28.0, 10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 6: {'distance': [278.0],

{2: {'distance': [20.0], 'drivable': [2100.0], 'road': [21.0], 'route': [21.0, 25.0, 5.0], 'speed': [8.0], 'start_time': [90.0], 't_ff': [104.208]}, 3: {'distance': [20.0], 'drivable': [1500.0], 'road': [15.0], 'route': [15.0], 'speed': [8.0], 'start_time': [91.0], 't_ff': [62.10000000000001]}, 5: {'distance': [287.0], 'drivable': [1201.0], 'road': [12.0], 'route': [12.0, 28.0, 10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 6: {'distance': [14.0], 'drivable': [1501.0], 'road': [15.0], 'route': [15.0], 'speed': [8.0], 'start_time': [151.0], 't_ff': [62.10000000000001]}, 7: {'distance': [238.0], 'drivable': [7701.0], 'road': [77.0], 'route': [77.0, 88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [165.0], 't_ff': [104.208]}, 8: {'distance': [322.0], 'drivable': [10501.0], 'road': [105.0], 'route': [105.0, 71.0, 78.0], 'speed': [0.0], 'start_time': [206.0], 't_ff': [77.976]}, 9: {'distance': [282.0], 'drivable': [1201.0], 'road': [12.0], 'route': [12.0, 28.0, 1

{2: {'distance': [134.0], 'drivable': [2501.0], 'road': [25.0], 'route': [25.0, 5.0], 'speed': [0.0], 'start_time': [90.0], 't_ff': [104.208]}, 5: {'distance': [213.0], 'drivable': [2800.0], 'road': [28.0], 'route': [28.0, 10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 7: {'distance': [238.0], 'drivable': [7701.0], 'road': [77.0], 'route': [77.0, 88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [165.0], 't_ff': [104.208]}, 8: {'distance': [241.11111111111117], 'drivable': [7100.0], 'road': [71.0], 'route': [71.0, 78.0], 'speed': [11.11111111111111], 'start_time': [206.0], 't_ff': [77.976]}, 9: {'distance': [208.00000000000006], 'drivable': [2800.0], 'road': [28.0], 'route': [28.0, 10.0, 34.0], 'speed': [0.0], 'start_time': [211.0], 't_ff': [70.68]}, 11: {'distance': [233.0], 'drivable': [7701.0], 'road': [77.0], 'route': [77.0, 88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [240.0], 't_ff': [104.208]}, 12: {'distance': [433.333333333333], 'drivable': 

{5: {'distance': [305.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 7: {'distance': [455.0], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [165.0], 't_ff': [104.208]}, 8: {'distance': [20.0], 'drivable': [7800.0], 'road': [78.0], 'route': [78.0], 'speed': [8.0], 'start_time': [206.0], 't_ff': [77.976]}, 9: {'distance': [300.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [211.0], 't_ff': [70.68]}, 11: {'distance': [450.00000000000006], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [240.0], 't_ff': [104.208]}, 12: {'distance': [228.0], 'drivable': [9800.0], 'road': [98.0], 'route': [98.0, 45.0, 44.0, 41.0], 'speed': [0.0], 'start_time': [250.0], 't_ff': [134.49]}, 14: {'distance': [354.0], 'drivable': [7100.0], 'road': [71.0], 'route': [71.0, 78.0], 'speed

{5: {'distance': [305.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 7: {'distance': [455.0], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [165.0], 't_ff': [104.208]}, 9: {'distance': [300.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [211.0], 't_ff': [70.68]}, 11: {'distance': [450.00000000000006], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [240.0], 't_ff': [104.208]}, 12: {'distance': [228.0], 'drivable': [9800.0], 'road': [98.0], 'route': [98.0, 45.0, 44.0, 41.0], 'speed': [0.0], 'start_time': [250.0], 't_ff': [134.49]}, 15: {'distance': [295.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [271.0], 't_ff': [70.68]}, 16: {'distance': [283.0], 'drivable': [9901.0], 'road': [99.0], 'route': [99.0, 15.0],

{5: {'distance': [305.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [151.0], 't_ff': [70.68]}, 7: {'distance': [455.0], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [165.0], 't_ff': [104.208]}, 9: {'distance': [300.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [211.0], 't_ff': [70.68]}, 11: {'distance': [450.00000000000006], 'drivable': [8801.0], 'road': [88.0], 'route': [88.0, 21.0, 25.0, 5.0], 'speed': [0.0], 'start_time': [240.0], 't_ff': [104.208]}, 12: {'distance': [228.0], 'drivable': [9800.0], 'road': [98.0], 'route': [98.0, 45.0, 44.0, 41.0], 'speed': [0.0], 'start_time': [250.0], 't_ff': [134.49]}, 15: {'distance': [295.0], 'drivable': [1001.0], 'road': [10.0], 'route': [10.0, 34.0], 'speed': [0.0], 'start_time': [271.0], 't_ff': [70.68]}, 16: {'distance': [241.11111111111117], 'drivable': [1500.0], 'road': [15.0], 'route': 

KeyboardInterrupt: 

In [386]:
scores = run_simulation(agent_specs, simulator_cfg_file, gym_cfg)

INFO:/starter-kit/train_dqn_example.py:

INFO:/starter-kit/train_dqn_example.py:****************************************


step  1   {42167350403: 3, 22372852612: 1, 42167350405: 3, 42294477575: 3, 42294463892: 1, 42167350420: 1, 14670355735: 1, 42381408549: 1, 14790105773: 2, 42167350445: 4, 42365406897: 2, 44052836274: 1, 12365406899: 2, 42167350456: 2, 22318148289: 2, 22318148293: 2, 42365409354: 2, 42167350476: 3, 42167350479: 2, 13987210067: 1, 42265990106: 2, 14454920703: 3}
step  2   {42167350403: 1, 22372852612: 1, 42167350405: 1, 42294477575: 2, 42294463892: 4, 42167350420: 2, 14670355735: 2, 42381408549: 4, 14790105773: 1, 42167350445: 4, 42365406897: 2, 44052836274: 2, 12365406899: 2, 42167350456: 3, 22318148289: 1, 22318148293: 1, 42365409354: 3, 42167350476: 4, 42167350479: 2, 13987210067: 1, 42265990106: 4, 14454920703: 2}
step  3   {42167350403: 4, 22372852612: 1, 42167350405: 3, 42294477575: 2, 42294463892: 1, 42167350420: 3, 14670355735: 3, 42381408549: 1, 14790105773: 3, 42167350445: 3, 42365406897: 4, 44052836274: 4, 12365406899: 4, 42167350456: 4, 22318148289: 3, 22318148293: 3, 4236540

step  24   {42167350403: 3, 22372852612: 4, 42167350405: 2, 42294477575: 3, 42294463892: 4, 42167350420: 2, 14670355735: 1, 42381408549: 1, 14790105773: 3, 42167350445: 1, 42365406897: 1, 44052836274: 1, 12365406899: 4, 42167350456: 1, 22318148289: 3, 22318148293: 4, 42365409354: 2, 42167350476: 1, 42167350479: 2, 13987210067: 4, 42265990106: 1, 14454920703: 4}
step  25   {42167350403: 2, 22372852612: 2, 42167350405: 2, 42294477575: 1, 42294463892: 3, 42167350420: 2, 14670355735: 4, 42381408549: 2, 14790105773: 4, 42167350445: 3, 42365406897: 1, 44052836274: 1, 12365406899: 2, 42167350456: 4, 22318148289: 3, 22318148293: 1, 42365409354: 3, 42167350476: 4, 42167350479: 2, 13987210067: 1, 42265990106: 3, 14454920703: 3}
step  26   {42167350403: 4, 22372852612: 1, 42167350405: 1, 42294477575: 1, 42294463892: 4, 42167350420: 4, 14670355735: 2, 42381408549: 2, 14790105773: 1, 42167350445: 3, 42365406897: 3, 44052836274: 3, 12365406899: 4, 42167350456: 2, 22318148289: 2, 22318148293: 4, 4236

step  48   {42167350403: 1, 22372852612: 1, 42167350405: 3, 42294477575: 3, 42294463892: 4, 42167350420: 3, 14670355735: 1, 42381408549: 4, 14790105773: 2, 42167350445: 4, 42365406897: 4, 44052836274: 4, 12365406899: 4, 42167350456: 1, 22318148289: 4, 22318148293: 1, 42365409354: 1, 42167350476: 2, 42167350479: 1, 13987210067: 4, 42265990106: 2, 14454920703: 4}
step  49   {42167350403: 2, 22372852612: 1, 42167350405: 2, 42294477575: 1, 42294463892: 1, 42167350420: 3, 14670355735: 3, 42381408549: 1, 14790105773: 4, 42167350445: 2, 42365406897: 4, 44052836274: 3, 12365406899: 2, 42167350456: 2, 22318148289: 1, 22318148293: 4, 42365409354: 1, 42167350476: 1, 42167350479: 3, 13987210067: 4, 42265990106: 1, 14454920703: 3}
step  50   {42167350403: 1, 22372852612: 4, 42167350405: 1, 42294477575: 1, 42294463892: 2, 42167350420: 3, 14670355735: 4, 42381408549: 4, 14790105773: 4, 42167350445: 2, 42365406897: 4, 44052836274: 4, 12365406899: 2, 42167350456: 2, 22318148289: 3, 22318148293: 1, 4236

KeyboardInterrupt: 